### Parte 2: Preparación de datos, generación de embeddings, indexación y almacenamiento en una base de datos vectorial

Este script realiza las siguientes tareas:
1. Carga y preprocesa los datos del dataset de tweets sobre clima.
2. Genera embeddings utilizando un modelo pre-entrenado.
3. Indexa y almacena los embeddings en una base de datos vectorial.
4. Demuestra cómo realizar consultas básicas en la base de datos vectorial.

In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
import chromadb
import pickle

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

c:\Users\andre\OneDrive\Desktop\Universidad Semestre 9\Almac y Recup de Info\trabajo2\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Usando dispositivo: cuda


In [2]:
# 1. Cargar y preprocesar los datos
print("Cargando y preprocesando datos...")
df = pd.read_csv('./data/processed/processedClimateTwitterData.csv')
texts = df['processed_text'].tolist()
ids = df.index.astype(str).tolist()

Cargando y preprocesando datos...


In [3]:
print("Generando embeddings...")
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)

def generate_embeddings_batch(texts, batch_size=64):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = tokenizer(batch, return_tensors="pt", truncation=True, max_length=512, padding=True)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = model(**inputs)
        batch_embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
        embeddings.extend(batch_embeddings)
        if i % 1000 == 0:
            print(f"Procesados {i}/{len(texts)} textos")
    return embeddings

# Generar embeddings en lotes
embeddings = generate_embeddings_batch(texts)

Generando embeddings...


c:\Users\andre\OneDrive\Desktop\Universidad Semestre 9\Almac y Recup de Info\trabajo2\myenv\lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
c:\Users\andre\OneDrive\Desktop\Universidad Semestre 9\Almac y Recup de Info\trabajo2\myenv\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Procesados 0/72405 textos
Procesados 8000/72405 textos
Procesados 16000/72405 textos
Procesados 24000/72405 textos
Procesados 32000/72405 textos
Procesados 40000/72405 textos
Procesados 48000/72405 textos
Procesados 56000/72405 textos
Procesados 64000/72405 textos
Procesados 72000/72405 textos


In [4]:
with open('./data/processed/embeddings.pkl', 'wb') as f:
    pickle.dump(embeddings, f)

In [5]:
# Cargar desde archivo pickle
with open('./data/processed/embeddings.pkl', 'rb') as f:
    embeddings = pickle.load(f)

In [6]:
# 3. Indexar y almacenar en una base de datos vectorial
print("Indexando y almacenando en ChromaDB...")

client = chromadb.PersistentClient(path="./data/processed/chroma_db")

collection = client.create_collection(name="climate_tweets")

# Dividimos los datos en lotes para evitar problemas de memoria
batch_size = 1000
for i in range(0, len(texts), batch_size):
    batch_texts = texts[i:i+batch_size]
    batch_embeddings = embeddings[i:i+batch_size]
    batch_ids = ids[i:i+batch_size]
    collection.add(
        embeddings=batch_embeddings,
        documents=batch_texts,
        ids=batch_ids
    )

print(f"Se han indexado {len(texts)} tweets en la base de datos vectorial.")

Indexando y almacenando en ChromaDB...
Se han indexado 72405 tweets en la base de datos vectorial.


In [7]:
collection = client.get_collection(name="climate_tweets")
print(collection)

Collection(id=5e0f7022-4dde-41c9-ab53-dc03af38ac20, name=climate_tweets)


In [10]:
# 4. Demostración de consulta
print("\nRealizando una consulta de ejemplo...")
query_text = [
    "Impact of climate change on biodiversity", # 0
    "Urbanization's impact on wildlife habitats", # 1
    "Ocean acidification and its consequences on marine life", # 2
    "Global warming and the melting of polar ice caps", # 3
    "Economic costs of climate change in coastal regions", # 4
    "Impact of agriculture on freshwater ecosystems", # 5
    "Relationship between carbon emissions and global temperature rise", # 6
    "Conservation strategies for endangered species in changing climates" # 7
]

# Generar embedding para la consulta
query_embedding = generate_embeddings_batch([query_text[5]])

# Realizar la consulta
results = collection.query(
    query_embeddings=[query_embedding],
    n_results=5
)

print("\nResultados de la consulta:")
for i, (id, text, distance) in enumerate(zip(results['ids'][0], results['documents'][0], results['distances'][0])):
    print(f"{i+1}. ID: {id}")
    print(f"   Text: {text}")
    print(f"   Distance: {distance}\n")

print("Proceso completado.")


Realizando una consulta de ejemplo...
Procesados 0/1 textos

Resultados de la consulta:
1. ID: 4995
   Text: jad daleycuenta_usuario regional ecosystem impact fr climatechange incl longer growing season snow rain altered soil moisture potl numero summer drought extreme event specie range shift invasive plant forest pest amp disease cuenta_usuario forestsaretheanswer
   Distance: 35.33061218261719

2. ID: 41153
   Text: soil crucial player climatechange mitigation agricultural resilience read new study lowcost reliable system measure soil organic carbon eventually increase link_url cuenta_usuario cuenta_usuario cuenta_usuario cuenta_usuario
   Distance: 36.292667388916016

3. ID: 8283
   Text: health ecosystem specie depend deteriorating rapidly ever ipbes 7cuenta_usuario globalassessment biodiversity ecosystem specie climatechange environment cuenta_usuario cuenta_usuario cuenta_usuario link_url
   Distance: 36.65413284301758

4. ID: 70377
   Text: health ecosystem specie depend deter